In [1]:
import os, sys
dir1 = os.path.abspath('..\..')
sys.path.append(dir1+ "\\")

import MetaTrader5 as mt
from datetime import datetime as dt

# custom
from my_jupyter.user import User
from my_jupyter.strategy_manager import StrategyManager
from my_jupyter.operation import Operation
from my_jupyter.strategies.min_max_strategy import MinMaxStrategy
from my_jupyter.market_data_repository import MarketDataRepository
# from Mbox import Mbox
# from boleta_1 import gerar_boleta
# from gerenciamento import Gerenciamento

## GUI's
from daemon_tools import DaemonThreadMini


In [2]:
strategies = [MinMaxStrategy()]
operation = Operation("WINV23", 1, mt.TIMEFRAME_M1, True, True)
user = User(strategies=strategies,operations=[operation], trading_time_start=dt.now())
market_data_repository = MarketDataRepository()
strategy = StrategyManager(market_data_repository, user)

# PARAMETROS MAIN
kwargs = {
    'strategy': strategy,
}

def main(**kwargs):
    _shift= 0 
    strategy: StrategyManager = kwargs['strategy']
    strategy.run_strategies()
    # print(f"{dt.now().strftime('%d/%m/%Y, %H:%M:%S')} >>> D-{_shift}")

In [3]:
def exec_daemon(**kwargs):
    main(**kwargs)
    # alarme.ring_alarme()   

In [4]:
# alarme = Alarme()
# alarme.set_alarme(True)
# alarme.filtrar_flag(sinais_abertura_daemon)
# alarme.filtrar_papeis(papeis_ibov_elevado )
# gerenciamento = Gerenciamento(mt, timeframe, timeframes_dict)

try:
    main(**kwargs)
except Exception as e:
    print(e.with_traceback(__tb=None))
    # Mbox.Alerta("ERRO", "busca de sinais com erro")


20/09/2023, 09:28:59 >>> D-0


In [ ]:
from my_jupyter.daemons.daemon_tools import DaemonThreadMini


thread_exec = DaemonThreadMini(exec_daemon, seconds=0, mseconds=100, **kwargs)
thread_exec.start()

20/09/2023, 09:28:59 >>> D-0


20/09/2023, 09:28:59 >>> D-0
SINAIS ATUALIZADOS
20/09/2023, 09:28:59 >>> D-0
SINAIS ATUALIZADOS
20/09/2023, 09:28:59 >>> D-0
SINAIS ATUALIZADOS


20/09/2023, 09:28:59 >>> D-0
SINAIS ATUALIZADOS
20/09/2023, 09:28:59 >>> D-0
SINAIS ATUALIZADOS
20/09/2023, 09:28:59 >>> D-0
SINAIS ATUALIZADOS
20/09/2023, 09:28:59 >>> D-0
SINAIS ATUALIZADOS
20/09/2023, 09:28:59 >>> D-0
SINAIS ATUALIZADOS
20/09/2023, 09:29:00 >>> D-0
SINAIS ATUALIZADOS
20/09/2023, 09:29:00 >>> D-0
SINAIS ATUALIZADOS
20/09/2023, 09:29:00 >>> D-0
SINAIS ATUALIZADOS
20/09/2023, 09:29:00 >>> D-0
SINAIS ATUALIZADOS
20/09/2023, 09:29:00 >>> D-0
SINAIS ATUALIZADOS
20/09/2023, 09:29:00 >>> D-0
SINAIS ATUALIZADOS
20/09/2023, 09:29:00 >>> D-0
SINAIS ATUALIZADOS
20/09/2023, 09:29:00 >>> D-0
SINAIS ATUALIZADOS
buying
1. order_send(): by WINV23 1.0 lots at 119210.0 with deviation=2 points
OrderSendResult(retcode=10009, deal=1208113850, order=1634060917, volume=1.0, price=119220.0, bid=119215.0, ask=119220.0, comment='Request executed', request_id=1346389351, retcode_external=0, request=TradeRequest(action=1, magic=1618, order=0, symbol='WINV23', volume=1.0, price=119210.0, stoplim